<a href="https://colab.research.google.com/github/ManagementBC/Organ/blob/main/stress_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!pip install web3
!pip install requests
!pip install ipfshttpclient



In [ ]:
!pip uninstall -y web3 eth-abi parsimonious



Found existing installation: web3 6.12.0
Uninstalling web3-6.12.0:
  Successfully uninstalled web3-6.12.0
Found existing installation: eth_abi 5.2.0
Uninstalling eth_abi-5.2.0:
  Successfully uninstalled eth_abi-5.2.0
Found existing installation: parsimonious 0.10.0
Uninstalling parsimonious-0.10.0:
  Successfully uninstalled parsimonious-0.10.0


In [ ]:
!pip install parsimonious==0.10.0 --quiet
!pip install web3==6.12.0 requests --quiet


In [ ]:
from web3 import Web3
import openai
import base64
import json
import requests


In [ ]:
from web3 import Web3
import openai
import base64
import json

# ───── Ethereum / Smart Contract Setup ─────
ETH_RPC_URL = "https://sepolia.infura.io/v3/xxxxxxxxxxxx"
PRIVATE_KEY = "xxxxxxxxxxxxxxxxxxxxxx"  # 🔐 Replace with your private key (never share)
SENDER_ADDRESS = Web3(Web3.HTTPProvider(ETH_RPC_URL)).eth.account.from_key(PRIVATE_KEY).address

CONTRACT_ADDRESS = "0x2B12f78430ad8DaDF1edcB20243A26aED8D0A3c9"  # 🔁 Replace with actual address

contract_abi = '''[{"inputs":[{"internalType":"address","name":"_regulator","type":"address"}],"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"uint256","name":"matchId","type":"uint256"},{"indexed":false,"internalType":"string","name":"approvedBy","type":"string"}],"name":"ApprovalGranted","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"donorAddress","type":"address"}],"name":"DonorAddressRegistered","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"uint256","name":"donorId","type":"uint256"},{"indexed":true,"internalType":"address","name":"donorAddress","type":"address"},{"indexed":false,"internalType":"string","name":"organType","type":"string"}],"name":"DonorRegistered","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"uint256","name":"id","type":"uint256"},{"indexed":false,"internalType":"string","name":"entityType","type":"string"}],"name":"EthicalApprovalGranted","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"committeeMember","type":"address"}],"name":"EthicalCommitteeMemberRegistered","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"llmAddress","type":"address"}],"name":"LLMRegistered","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"uint256","name":"matchId","type":"uint256"},{"indexed":true,"internalType":"uint256","name":"donorId","type":"uint256"},{"indexed":true,"internalType":"uint256","name":"recipientId","type":"uint256"},{"indexed":false,"internalType":"address","name":"llm","type":"address"}],"name":"MatchRecorded","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"recipientAddress","type":"address"}],"name":"RecipientAddressRegistered","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"uint256","name":"recipientId","type":"uint256"},{"indexed":true,"internalType":"address","name":"recipientAddress","type":"address"},{"indexed":false,"internalType":"string","name":"organType","type":"string"}],"name":"RecipientRegistered","type":"event"},{"inputs":[{"internalType":"uint256","name":"matchId","type":"uint256"}],"name":"approveDonor","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"uint256","name":"donorId","type":"uint256"}],"name":"approveDonorEthicalCommittee","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"uint256","name":"matchId","type":"uint256"}],"name":"approveFinalTransplant","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"uint256","name":"matchId","type":"uint256"}],"name":"approveHospital","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"uint256","name":"matchId","type":"uint256"}],"name":"approveMedicalTeam","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"uint256","name":"matchId","type":"uint256"}],"name":"approveRecipient","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"uint256","name":"recipientId","type":"uint256"}],"name":"approveRecipientEthicalCommittee","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"","type":"address"}],"name":"authorizedLLMs","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint256","name":"donorId","type":"uint256"},{"internalType":"uint256","name":"recipientId1","type":"uint256"},{"internalType":"uint256","name":"recipientId2","type":"uint256"}],"name":"createMatch","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"donorCounter","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint256","name":"","type":"uint256"}],"name":"donors","outputs":[{"internalType":"uint256","name":"donorId","type":"uint256"},{"internalType":"address","name":"donorAddress","type":"address"},{"internalType":"string","name":"bloodType","type":"string"},{"internalType":"string","name":"hlaTyping","type":"string"},{"internalType":"string","name":"organType","type":"string"},{"internalType":"string","name":"ipfsHash","type":"string"},{"internalType":"bool","name":"registered","type":"bool"},{"internalType":"bool","name":"ethicalApproved","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint256","name":"matchId","type":"uint256"}],"name":"isTransplantApproved","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"matchCounter","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint256","name":"","type":"uint256"}],"name":"matches","outputs":[{"internalType":"uint256","name":"matchId","type":"uint256"},{"internalType":"uint256","name":"donorId","type":"uint256"},{"internalType":"uint256","name":"recipientId","type":"uint256"},{"internalType":"address","name":"matchedByLLM","type":"address"},{"internalType":"bool","name":"medicalApproved","type":"bool"},{"internalType":"bool","name":"hospitalApproved","type":"bool"},{"internalType":"bool","name":"donorApproved","type":"bool"},{"internalType":"bool","name":"recipientApproved","type":"bool"},{"internalType":"bool","name":"ethicalCommitteeApproved","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"recipientCounter","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint256","name":"","type":"uint256"}],"name":"recipients","outputs":[{"internalType":"uint256","name":"recipientId","type":"uint256"},{"internalType":"address","name":"recipientAddress","type":"address"},{"internalType":"string","name":"bloodType","type":"string"},{"internalType":"string","name":"hlaTyping","type":"string"},{"internalType":"string","name":"organType","type":"string"},{"internalType":"string","name":"ipfsHash","type":"string"},{"internalType":"bool","name":"registered","type":"bool"},{"internalType":"bool","name":"matched","type":"bool"},{"internalType":"bool","name":"ethicalApproved","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"donorAddress","type":"address"},{"internalType":"string","name":"bloodType","type":"string"},{"internalType":"string","name":"hlaTyping","type":"string"},{"internalType":"string","name":"organType","type":"string"},{"internalType":"string","name":"ipfsHash","type":"string"}],"name":"registerDonor","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"donorAddress","type":"address"}],"name":"registerDonorAddress","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"committeeMember","type":"address"}],"name":"registerEthicalCommittee","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"hospital","type":"address"}],"name":"registerHospital","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"llm","type":"address"}],"name":"registerLLM","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"medicalTeam","type":"address"}],"name":"registerMedicalTeam","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"recipientAddress","type":"address"},{"internalType":"string","name":"bloodType","type":"string"},{"internalType":"string","name":"hlaTyping","type":"string"},{"internalType":"string","name":"organType","type":"string"},{"internalType":"string","name":"ipfsHash","type":"string"}],"name":"registerRecipient","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"recipientAddress","type":"address"}],"name":"registerRecipientAddress","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"","type":"address"}],"name":"registeredDonorAddresses","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"","type":"address"}],"name":"registeredEthicalCommittee","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"","type":"address"}],"name":"registeredHospitals","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"","type":"address"}],"name":"registeredMedicalTeams","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"","type":"address"}],"name":"registeredRecipientAddresses","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"regulator","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"}]'''  # Replace with the contract ABI as a string


w3 = Web3(Web3.HTTPProvider(ETH_RPC_URL))
contract = w3.eth.contract(address=CONTRACT_ADDRESS, abi=contract_abi)

# ───── OpenAI Fine-tuned Model Setup ─────
openai.api_key = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"  # ✅ Your API key
MODEL_ID = "ft:gpt-4o-mini-2024-07-18:personal:organdonations:AmbvC8Vu"

# ───── IPFS (via Pinata) Setup ─────
pinata_api_key = "xxxxxxxxx"
pinata_secret_api_key = "xxxxxxxxxxxxxxxxxx"

def fetch_patient_data_from_ipfs(patient_id: int):
    """
    Fetches donor IPFS data by calling the smart contract and accessing via Pinata.
    """
    try:
        # Get IPFS hash from `donors` mapping
        donor_data = contract.functions.donors(patient_id).call()
        ipfs_hash = donor_data[5]  # index 5 = ipfsHash in the Donor struct
    except Exception as e:
        print(f"❌ Error retrieving IPFS hash from contract: {e}")
        return None

    try:
        url = f"https://gateway.pinata.cloud/ipfs/{ipfs_hash}"
        headers = {
            "pinata_api_key": pinata_api_key,
            "pinata_secret_api_key": pinata_secret_api_key
        }
        response = requests.get(url, headers=headers, timeout=60)
        if response.status_code == 200:
            print(f"✅ Retrieved data for donor {patient_id}")
            return response.json()
        else:
            print(f"❌ IPFS fetch failed: {response.status_code} - {response.text}")
            return None
    except Exception as e:
        print(f"❌ Error fetching from Pinata: {e}")
        return None


In [ ]:
contract = w3.eth.contract(address=CONTRACT_ADDRESS, abi=contract_abi)


In [ ]:
import time
from concurrent.futures import ThreadPoolExecutor, as_completed

# --- On-Chain: matchDonor Transaction ---
def send_match_donor_tx(donor_id: str):
    """
    Calls createMatch on the contract (with the same recipient used twice if only 1 exists).
    """
    try:
        nonce = w3.eth.get_transaction_count(SENDER_ADDRESS)
        tx = contract.functions.createMatch(
            int(donor_id),   # donor ID
            1,               # recipientId1 (same)
            1                # recipientId2 (same, due to 1 recipient only)
        ).build_transaction({
            "from": SENDER_ADDRESS,
            "nonce": nonce,
            "gas": 400000,
            "maxFeePerGas": w3.to_wei(30, 'gwei'),
            "maxPriorityFeePerGas": w3.to_wei(2, 'gwei')
        })
        signed_tx = w3.eth.account.sign_transaction(tx, private_key=PRIVATE_KEY)
        start_time = time.time()
        tx_hash = w3.eth.send_raw_transaction(signed_tx.rawTransaction)
        receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
        end_time = time.time()
        return {
            "donor_id": donor_id,
            "tx_hash": tx_hash.hex(),
            "receipt_time": end_time - start_time,
            "gas_used": receipt.gasUsed
        }
    except Exception as e:
        return {"donor_id": donor_id, "error": str(e)}

# --- IPFS: Fetch Patient Data ---
def fetch_ipfs(patient_index: int):
    """
    Uses the smart contract to get the IPFS hash and fetches the data from Pinata.
    """
    try:
        start_time = time.time()
        result = fetch_patient_data_from_ipfs(patient_index)
        end_time = time.time()
        if result:
            return {
                "patient_index": patient_index,
                "ipfs_data": result,
                "response_time": end_time - start_time
            }
        else:
            return {"patient_index": patient_index, "error": "Failed to retrieve or decode IPFS JSON."}
    except Exception as e:
        return {"patient_index": patient_index, "error": str(e)}

# --- Full Stress Test Runner ---
def full_pipeline_stress_test(patient_ids, max_workers=5, do_onchain=True, do_offchain=True, do_ipfs=True):
    """
    Executes full stress test: blockchain matchDonor + LLM call + IPFS fetch in parallel.
    """
    print(f"🚀 Starting stress test with {len(patient_ids)} patients using {max_workers} threads...\n")
    results = []

    def task(pid):
        res = {"patient_id": pid}
        if do_onchain:
            res["onchain"] = send_match_donor_tx(str(pid))
        if do_offchain:
            res["llm"] = call_llm_matcher(str(pid))
        if do_ipfs:
            res["ipfs"] = fetch_ipfs(pid)
        return res

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = [executor.submit(task, pid) for pid in patient_ids]
        for fut in as_completed(futures):
            result = fut.result()
            results.append(result)
            print(f"✅ Completed: {result['patient_id']}")

    print("\n📊 Test Summary:")
    onchain_times = [r["onchain"]["receipt_time"] for r in results if "onchain" in r and "receipt_time" in r["onchain"]]
    llm_times = [r["llm"]["response_time"] for r in results if "llm" in r and "response_time" in r["llm"]]
    ipfs_times = [r["ipfs"]["response_time"] for r in results if "ipfs" in r and "response_time" in r["ipfs"]]

    if onchain_times:
        print(f"⏱ Average On-Chain Time: {sum(onchain_times)/len(onchain_times):.2f} sec")
    if llm_times:
        print(f"⏱ Average LLM Time: {sum(llm_times)/len(llm_times):.2f} sec")
    if ipfs_times:
        print(f"⏱ Average IPFS Fetch Time: {sum(ipfs_times)/len(ipfs_times):.2f} sec")

    return results



In [ ]:
import openai
openai.api_key = "xxxxxxxxxxxxxxxxxxxx"

def call_llm_matcher(donor_id: str):
    try:
        start_time = time.time()
        response = openai.ChatCompletion.create(
            model="gpt-4",  # or your fine-tuned model ID
            messages=[
                {"role": "system", "content": "You are an expert in organ donor-recipient matching."},
                {"role": "user", "content": f"Find a match for donor ID: {donor_id}"}
            ]
        )
        end_time = time.time()
        return {
            "donor_id": donor_id,
            "llm_output": response.choices[0].message["content"].strip(),
            "response_time": end_time - start_time
        }
    except Exception as e:
        return {"donor_id": donor_id, "error": str(e)}



In [ ]:
results = full_pipeline_stress_test(patient_ids=[1], max_workers=1)


In [ ]:
results = full_pipeline_stress_test(patient_ids=list(range(1, 51)), max_workers=10)

In [ ]:
pip install openai==0.28
